In [6]:
import os
import csv
import pandas as pd
import time
import pickle
import sys
import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from collections import defaultdict


In [2]:
print(os.getcwd())
# it should end with this: /AITutor_SeqModeling
# if not, run the next block


/local-scratch/localhome/pagand/projects/mygitsDaTu/AITutor_SeqModeling/KnowledgeTracking


In [3]:
# run if the current directory is not AITutor_SeqModeling
cwd = os.chdir(os.path.join(os.getcwd(), ".."))
print(os.getcwd())

/local-scratch/localhome/pagand/projects/mygitsDaTu/AITutor_SeqModeling


In [4]:
File_pickle = "data/KT_logs_annotated.pkl"

# read from pickle
df = pd.read_pickle(File_pickle)

df.head()

,username,skill,correct,time
0,a1,"[Supervised Learning, Classification Algorithms]",True,0.000000
1,a1,"[Supervised Learning, Classification Algorithms]",False,3.726700
2,a2,"[Supervised Learning, Classification Algorithms]",False,0.000000
3,a2,"[Supervised Learning, Classification Algorithms]",True,1.987467
4,a3,"[Supervised Learning, Classification Algorithms]",True,0.000000


In [10]:
skills = pickle.load(open("data/Skill_hirereachy.pkl", "rb"))

In [ ]:
# version 1: Only user params
# user_params = {}

# def initialize_user_params(user_id, skills):
#     user_params[user_id] = {}
#     for skill in skills.keys():
#         skill_params = skills[skill][-1]
#         user_params[user_id][skill] = {
#             "P(L)": skill_params[0],
#             "P(T)": skill_params[1],
#             "P(G)": skill_params[2],
#             "P(S)": skill_params[3]
#         }
# for user_id in df["username"].unique():
#     initialize_user_params(user_id, skills)

In [18]:
# version 2: seperate user-specific and skill specific parameters
def initialize_params(skills, user_ids):
    user_params = {}
    skill_params = {}
    for skill in skills.keys():
        skill_params[skill] = {
                "P(L)": skills[skill][-1][0],
                "P(T)": skills[skill][-1][1],
                "P(G)": skills[skill][-1][1],
                "P(S)": skills[skill][-1][3]
            }
        for user_id in user_ids:
            # assume all users have the same initial skill level
            # add prior knowledge here if exists
            user_params[user_id] = skill_params
    return skill_params, user_params

In [11]:
# Function to compute P(C_t|L_t, G, S)
def compute_prob_correctness(P_L, P_G, P_S, correct):
    if correct:
        return (1 - P_S) * P_L + P_G * (1 - P_L)
    else:
        return P_S * P_L + (1 - P_G) * (1 - P_L)

In [12]:
# (log-likelihood function)
def log_likelihood(interaction_log,  skill_params, user_params):
    log_likelihood = 0
    for _, row in interaction_log.iterrows():
        user_id = row["username"]
        skill_list = row["skill"]
        correctness = row["correct"]

        for skill in skill_list:
            # Retrieve user and skill parameters
            P_L = user_params[user_id][skill]["P(L)"]
            P_G = skill_params[skill]["P(G)"]
            P_S = skill_params[skill]["P(S)"]

            # Compute likelihood
            prob = compute_prob_correctness(P_L, P_G, P_S, correctness)
            log_likelihood += np.log(prob + 1e-9)

    return -log_likelihood

In [21]:
# Compute gradients
def compute_gradients(interaction_log, skill_params, user_params, skills):
    skill_grads = defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0})
    user_grads = defaultdict(lambda: defaultdict(lambda: {"P(L)": 0}))

    for _, row in interaction_log.iterrows():
        user_id = row["username"]
        skills_list = row["skill"]
        correctness = row["correct"]

        for skill in skills_list:
            P_L_user = user_params[user_id][skill]["P(L)"]
            P_L_skill = skill_params[skill]["P(L)"]
            P_T = skill_params[skill]["P(T)"]
            P_G = skill_params[skill]["P(G)"]
            P_S = skill_params[skill]["P(S)"]

            prob = compute_prob_correctness(P_L_user, P_G, P_S, correctness)

            # Gradient for user-specific P(L)
            grad_user = (correctness - prob) / (prob + 1e-9)
            user_grads[user_id][skill]["P(L)"] += grad_user

            # Gradients for skill-specific parameters
            grad_L = (correctness - prob) * (1 - P_S - P_G)
            grad_T = 0  # Placeholder for transition term
            grad_G = (correctness - prob) * (1 - P_L_user)
            grad_S = (correctness - prob) * P_L_user

            skill_grads[skill]["P(L)"] += grad_L
            skill_grads[skill]["P(T)"] += grad_T
            skill_grads[skill]["P(G)"] += grad_G
            skill_grads[skill]["P(S)"] += grad_S

    return skill_grads, user_grads

In [22]:
# Update parameters with SGD or Adam
def update_params(skill_params, user_params, skill_grads, user_grads, learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-8, t=1):
    # Adam accumulators
    skill_m = defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0})
    skill_v = defaultdict(lambda: {"P(L)": 0, "P(T)": 0, "P(G)": 0, "P(S)": 0})
    user_m = defaultdict(lambda: defaultdict(lambda: {"P(L)": 0}))
    user_v = defaultdict(lambda: defaultdict(lambda: {"P(L)": 0}))

    # Skill-specific updates
    for skill, grads in skill_grads.items():
        for param, grad in grads.items():
            m = skill_m[skill][param]
            v = skill_v[skill][param]

            m = beta1 * m + (1 - beta1) * grad
            v = beta2 * v + (1 - beta2) * (grad ** 2)
            m_hat = m / (1 - beta1 ** t)
            v_hat = v / (1 - beta2 ** t)

            skill_params[skill][param] -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
            skill_m[skill][param] = m
            skill_v[skill][param] = v

    # User-specific updates
    for user_id, user_skills in user_grads.items():
        for skill, grads in user_skills.items():
            for param, grad in grads.items():
                m = user_m[user_id][skill][param]
                v = user_v[user_id][skill][param]

                m = beta1 * m + (1 - beta1) * grad
                v = beta2 * v + (1 - beta2) * (grad ** 2)
                m_hat = m / (1 - beta1 ** t)
                v_hat = v / (1 - beta2 ** t)

                user_params[user_id][skill][param] -= learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
                user_m[user_id][skill][param] = m
                user_v[user_id][skill][param] = v

    return skill_params, user_params

In [30]:
# Parent-Child Constraints
def enforce_constraints(user_params, skill_params, skills):
    for skill, skill_data in skills.items():
        parents = skill_data[1]
        for parent in parents:
            if skill_params[skill]["P(L)"] >= skill_params[parent]["P(L)"]:
                skill_params[skill]["P(L)"] = skill_params[parent]["P(L)"] - 0.01 
            for user in user_params:
                parent_prob = user_params[user][parent]["P(L)"]
                child_prob = user_params[user][skill]["P(L)"]
                if child_prob >= parent_prob:
                    user_params[user][skill]["P(L)"] = parent_prob - 0.01  # Apply heuristic

In [27]:
def run_em_with_sgd(interaction_log, skills, max_iter=50, learning_rate=0.01, reg_lambda=0.1):
    skill_params, user_params = initialize_params(skills, interaction_log["username"].unique())

    for t in range(1, max_iter + 1):
        print(f"Iteration {t}")

        # Compute gradients
        skill_grads, user_grads = compute_gradients(interaction_log, skill_params, user_params, skills)

        # Update parameters
        skill_params, user_params = update_params(skill_params, user_params, skill_grads, user_grads,
                                                  learning_rate=learning_rate, t=t)

        # Enforce constraints
        enforce_constraints(user_params, skill_params, skills)

        # Log likelihood for monitoring
        likelihood = -log_likelihood(interaction_log, skill_params, user_params)
        print(f"  Log-Likelihood: {likelihood:.4f}")

    return skill_params, user_params

In [36]:
# Run the EM Algorithm
skill_params, user_params = run_em_with_sgd(df.iloc[:1000], skills)

# Output Results
print("Final Skill Parameters:")
for skill, params in skill_params.items():
    print(skill, params)

print("\nFinal User Parameters:")
for user_id, user_data in user_params.items():
    print(user_id)
    for skill, params in user_data.items():
        print(f"  {skill}: {params}")

Iteration 1


/tmp/ipykernel_3696861/3475367163.py:17: RuntimeWarning: invalid value encountered in log
  log_likelihood += np.log(prob + 1e-9)


  Log-Likelihood: nan
Iteration 2
  Log-Likelihood: nan
Iteration 3
  Log-Likelihood: -1135.5528
Iteration 4
  Log-Likelihood: nan
Iteration 5
  Log-Likelihood: nan
Iteration 6
  Log-Likelihood: nan
Iteration 7
  Log-Likelihood: nan
Iteration 8
  Log-Likelihood: nan
Iteration 9
  Log-Likelihood: nan
Iteration 10
  Log-Likelihood: nan
Iteration 11
  Log-Likelihood: nan
Iteration 12
  Log-Likelihood: nan
Iteration 13
  Log-Likelihood: nan
Iteration 14
  Log-Likelihood: nan
Iteration 15
  Log-Likelihood: nan
Iteration 16
  Log-Likelihood: nan
Iteration 17
  Log-Likelihood: nan
Iteration 18
  Log-Likelihood: nan
Iteration 19
  Log-Likelihood: nan
Iteration 20
  Log-Likelihood: nan
Iteration 21
  Log-Likelihood: nan
Iteration 22
  Log-Likelihood: nan
Iteration 23
  Log-Likelihood: nan
Iteration 24
  Log-Likelihood: nan
Iteration 25
  Log-Likelihood: nan
Iteration 26
  Log-Likelihood: nan
Iteration 27
  Log-Likelihood: nan
Iteration 28
  Log-Likelihood: nan
Iteration 29
  Log-Likelihood: nan